## GANS
Source: https://medium.com/@mattiaspinelli/simple-generative-adversarial-network-gans-with-keras-1fe578e44a87

In [1]:
# -*- coding: utf-8 -*-
""" Simple implementation of Generative Adversarial Neural Network """
import os
import numpy as np

from IPython.core.debugger import Tracer

from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization
from keras.layers.advanced_activations import LeakyReLU
from keras.models import Sequential
from keras.optimizers import Adam

import matplotlib.pyplot as plt
plt.switch_backend('agg')   # allows code to run without a system DISPLAY


class GAN(object):
    """ Generative Adversarial Network class """
    def __init__(self, width=28, height=28, channels=1):

        self.width = width
        self.height = height
        self.channels = channels

        self.shape = (self.width, self.height, self.channels)

        self.optimizer = Adam(lr=0.0002, beta_1=0.5, decay=8e-8)

        self.G = self.__generator()
        self.G.compile(loss='binary_crossentropy', optimizer=self.optimizer)

        self.D = self.__discriminator()
        self.D.compile(loss='binary_crossentropy', optimizer=self.optimizer, metrics=['accuracy'])

        self.stacked_generator_discriminator = self.__stacked_generator_discriminator()

        self.stacked_generator_discriminator.compile(loss='binary_crossentropy', optimizer=self.optimizer)


    def __generator(self):
        """ Declare generator """

        model = Sequential()
        model.add(Dense(256, input_shape=(100,)))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(1024))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(self.width  * self.height * self.channels, activation='tanh'))
        model.add(Reshape((self.width, self.height, self.channels)))

        return model

    def __discriminator(self):
        """ Declare discriminator """

        model = Sequential()
        model.add(Flatten(input_shape=self.shape))
        model.add(Dense((self.width * self.height * self.channels), input_shape=self.shape))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(np.int64((self.width * self.height * self.channels)/2)))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(1, activation='sigmoid'))
        model.summary()

        return model

    def __stacked_generator_discriminator(self):
        """GAN composed of generator and discriminator!!"""
        self.D.trainable = False ## this freezes the weight of discriminator
        ## D does not learn in this stage
        ## in this stage G learns how to fool D
        ## G updates its output such that 

        model = Sequential()
        model.add(self.G)
        model.add(self.D)

        return model

    def train(self, X_train, epochs=20000, batch = 32, save_interval = 100):

        for cnt in range(epochs):

            ## train discriminator
            random_index = np.random.randint(0, len(X_train) - np.int64(batch/2))
            legit_images = X_train[random_index : random_index + np.int64(batch/2)].reshape(np.int64(batch/2), self.width, self.height, self.channels)

            gen_noise = np.random.normal(0, 1, (np.int64(batch/2), 100))
            syntetic_images = self.G.predict(gen_noise)

            x_combined_batch = np.concatenate((legit_images, syntetic_images))
            y_combined_batch = np.concatenate((np.ones((np.int64(batch/2), 1)), np.zeros((np.int64(batch/2), 1))))

            d_loss = self.D.train_on_batch(x_combined_batch, y_combined_batch)


            # train generator
                 # Here we dont train the generator directly. 
                 # We train it indirectly through the adversarial model.
                 # Notice that in this stage self.D.trainable = False (weights of D are frozen)
            noise = np.random.normal(0, 1, (batch, 100))
            y_mislabled = np.ones((batch, 1))
            g_loss = self.stacked_generator_discriminator.train_on_batch(noise, y_mislabled)

            print ('epoch: %d, [Discriminator :: d_loss: %f], [ Generator :: loss: %f]' % (cnt, d_loss[0], g_loss))
            if cnt % save_interval == 0:
                self.plot_images(save2file=True, step=cnt)


    def plot_images(self, save2file=False, samples=16, step=0):
        ''' Plot and generated images '''
        if not os.path.exists("./images"):
            os.makedirs("./images")
        filename = "./images/mnist_%d.png" % step
        noise = np.random.normal(0, 1, (samples, 100))

        images = self.G.predict(noise)

        plt.figure(figsize=(10, 10))

        for i in range(images.shape[0]):
            plt.subplot(4, 4, i+1)
            image = images[i, :, :, :]
            image = np.reshape(image, [self.height, self.width])
            plt.imshow(image, cmap='gray')
            plt.axis('off')
        plt.tight_layout()

        if save2file:
            plt.savefig(filename)
            plt.close('all')
        else:
            plt.show()


if __name__ == '__main__':
    (X_train, _), (_, _) = mnist.load_data()

    # Rescale -1 to 1
    X_train = (X_train.astype(np.float32) - 127.5) / 127.5
    X_train = np.expand_dims(X_train, axis=3)


    gan = GAN()
    gan.train(X_train)

Using TensorFlow backend.


Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 784)               615440    
_________________________________________________________________
leaky_re_lu_4 (LeakyReLU)    (None, 784)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 392)               307720    
_________________________________________________________________
leaky_re_lu_5 (LeakyReLU)    (None, 392)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 393    

/anaconda3/lib/python3.6/site-packages/keras/engine/training.py:973: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


epoch: 0, [Discriminator :: d_loss: 0.743550], [ Generator :: loss: 0.743447]
epoch: 1, [Discriminator :: d_loss: 0.412380], [ Generator :: loss: 0.741138]
epoch: 2, [Discriminator :: d_loss: 0.375833], [ Generator :: loss: 0.790600]
epoch: 3, [Discriminator :: d_loss: 0.348111], [ Generator :: loss: 0.879832]
epoch: 4, [Discriminator :: d_loss: 0.311981], [ Generator :: loss: 1.144351]
epoch: 5, [Discriminator :: d_loss: 0.276222], [ Generator :: loss: 1.300179]
epoch: 6, [Discriminator :: d_loss: 0.224711], [ Generator :: loss: 1.600458]
epoch: 7, [Discriminator :: d_loss: 0.164070], [ Generator :: loss: 1.860233]
epoch: 8, [Discriminator :: d_loss: 0.140815], [ Generator :: loss: 2.152984]
epoch: 9, [Discriminator :: d_loss: 0.107452], [ Generator :: loss: 2.387309]
epoch: 10, [Discriminator :: d_loss: 0.072351], [ Generator :: loss: 2.613440]
epoch: 11, [Discriminator :: d_loss: 0.066371], [ Generator :: loss: 2.775857]
epoch: 12, [Discriminator :: d_loss: 0.044956], [ Generator ::

epoch: 107, [Discriminator :: d_loss: 0.004949], [ Generator :: loss: 5.176146]
epoch: 108, [Discriminator :: d_loss: 0.009958], [ Generator :: loss: 5.293435]
epoch: 109, [Discriminator :: d_loss: 0.007007], [ Generator :: loss: 5.085060]
epoch: 110, [Discriminator :: d_loss: 0.006962], [ Generator :: loss: 5.068310]
epoch: 111, [Discriminator :: d_loss: 0.011367], [ Generator :: loss: 5.405216]
epoch: 112, [Discriminator :: d_loss: 0.008237], [ Generator :: loss: 5.368801]
epoch: 113, [Discriminator :: d_loss: 0.009146], [ Generator :: loss: 5.451484]
epoch: 114, [Discriminator :: d_loss: 0.010954], [ Generator :: loss: 5.571823]
epoch: 115, [Discriminator :: d_loss: 0.007945], [ Generator :: loss: 5.305620]
epoch: 116, [Discriminator :: d_loss: 0.018341], [ Generator :: loss: 5.522955]
epoch: 117, [Discriminator :: d_loss: 0.011700], [ Generator :: loss: 5.404444]
epoch: 118, [Discriminator :: d_loss: 0.009319], [ Generator :: loss: 5.636800]
epoch: 119, [Discriminator :: d_loss: 0.

...

epoch: 19501, [Discriminator :: d_loss: 0.596875], [ Generator :: loss: 0.865905]
epoch: 19502, [Discriminator :: d_loss: 0.637384], [ Generator :: loss: 0.832069]
epoch: 19503, [Discriminator :: d_loss: 0.695172], [ Generator :: loss: 0.792672]
epoch: 19504, [Discriminator :: d_loss: 0.747816], [ Generator :: loss: 0.727573]
epoch: 19505, [Discriminator :: d_loss: 0.642864], [ Generator :: loss: 0.794578]
epoch: 19506, [Discriminator :: d_loss: 0.571248], [ Generator :: loss: 0.843159]
epoch: 19507, [Discriminator :: d_loss: 0.743697], [ Generator :: loss: 0.861094]
epoch: 19508, [Discriminator :: d_loss: 0.637004], [ Generator :: loss: 0.934902]
epoch: 19509, [Discriminator :: d_loss: 0.711248], [ Generator :: loss: 0.827325]
epoch: 19510, [Discriminator :: d_loss: 0.758768], [ Generator :: loss: 0.803923]
epoch: 19511, [Discriminator :: d_loss: 0.653049], [ Generator :: loss: 0.772914]
epoch: 19512, [Discriminator :: d_loss: 0.713530], [ Generator :: loss: 0.820534]
epoch: 19513, [D

epoch: 19601, [Discriminator :: d_loss: 0.657790], [ Generator :: loss: 1.020855]
epoch: 19602, [Discriminator :: d_loss: 0.638661], [ Generator :: loss: 0.997301]
epoch: 19603, [Discriminator :: d_loss: 0.668595], [ Generator :: loss: 0.909742]
epoch: 19604, [Discriminator :: d_loss: 0.627624], [ Generator :: loss: 0.836151]
epoch: 19605, [Discriminator :: d_loss: 0.595026], [ Generator :: loss: 0.847488]
epoch: 19606, [Discriminator :: d_loss: 0.646609], [ Generator :: loss: 0.767408]
epoch: 19607, [Discriminator :: d_loss: 0.771390], [ Generator :: loss: 0.658282]
epoch: 19608, [Discriminator :: d_loss: 0.649557], [ Generator :: loss: 0.784405]
epoch: 19609, [Discriminator :: d_loss: 0.626443], [ Generator :: loss: 0.849369]
epoch: 19610, [Discriminator :: d_loss: 0.643471], [ Generator :: loss: 0.984957]
epoch: 19611, [Discriminator :: d_loss: 0.731422], [ Generator :: loss: 0.963001]
epoch: 19612, [Discriminator :: d_loss: 0.600983], [ Generator :: loss: 0.880969]
epoch: 19613, [D

epoch: 19701, [Discriminator :: d_loss: 0.566662], [ Generator :: loss: 1.016965]
epoch: 19702, [Discriminator :: d_loss: 0.708674], [ Generator :: loss: 0.877470]
epoch: 19703, [Discriminator :: d_loss: 0.595548], [ Generator :: loss: 0.656628]
epoch: 19704, [Discriminator :: d_loss: 0.566323], [ Generator :: loss: 0.745276]
epoch: 19705, [Discriminator :: d_loss: 0.552313], [ Generator :: loss: 0.839375]
epoch: 19706, [Discriminator :: d_loss: 0.740156], [ Generator :: loss: 1.014870]
epoch: 19707, [Discriminator :: d_loss: 0.741060], [ Generator :: loss: 0.698455]
epoch: 19708, [Discriminator :: d_loss: 0.684522], [ Generator :: loss: 0.775300]
epoch: 19709, [Discriminator :: d_loss: 0.618243], [ Generator :: loss: 0.785146]
epoch: 19710, [Discriminator :: d_loss: 0.632573], [ Generator :: loss: 0.777072]
epoch: 19711, [Discriminator :: d_loss: 0.724620], [ Generator :: loss: 0.856088]
epoch: 19712, [Discriminator :: d_loss: 0.623374], [ Generator :: loss: 0.883872]
epoch: 19713, [D

epoch: 19801, [Discriminator :: d_loss: 0.685965], [ Generator :: loss: 0.710926]
epoch: 19802, [Discriminator :: d_loss: 0.694334], [ Generator :: loss: 0.690681]
epoch: 19803, [Discriminator :: d_loss: 0.724818], [ Generator :: loss: 0.891825]
epoch: 19804, [Discriminator :: d_loss: 0.691066], [ Generator :: loss: 0.860667]
epoch: 19805, [Discriminator :: d_loss: 0.681737], [ Generator :: loss: 0.855227]
epoch: 19806, [Discriminator :: d_loss: 0.655102], [ Generator :: loss: 0.844939]
epoch: 19807, [Discriminator :: d_loss: 0.604247], [ Generator :: loss: 0.919469]
epoch: 19808, [Discriminator :: d_loss: 0.583879], [ Generator :: loss: 0.818100]
epoch: 19809, [Discriminator :: d_loss: 0.585244], [ Generator :: loss: 0.900580]
epoch: 19810, [Discriminator :: d_loss: 0.609944], [ Generator :: loss: 0.690424]
epoch: 19811, [Discriminator :: d_loss: 0.651541], [ Generator :: loss: 0.852866]
epoch: 19812, [Discriminator :: d_loss: 0.598272], [ Generator :: loss: 0.965703]
epoch: 19813, [D

epoch: 19901, [Discriminator :: d_loss: 0.606836], [ Generator :: loss: 0.814248]
epoch: 19902, [Discriminator :: d_loss: 0.593190], [ Generator :: loss: 0.737503]
epoch: 19903, [Discriminator :: d_loss: 0.656585], [ Generator :: loss: 0.742774]
epoch: 19904, [Discriminator :: d_loss: 0.620498], [ Generator :: loss: 0.894430]
epoch: 19905, [Discriminator :: d_loss: 0.656074], [ Generator :: loss: 0.858799]
epoch: 19906, [Discriminator :: d_loss: 0.711041], [ Generator :: loss: 0.860798]
epoch: 19907, [Discriminator :: d_loss: 0.608714], [ Generator :: loss: 1.023895]
epoch: 19908, [Discriminator :: d_loss: 0.744765], [ Generator :: loss: 0.898499]
epoch: 19909, [Discriminator :: d_loss: 0.671190], [ Generator :: loss: 0.710761]
epoch: 19910, [Discriminator :: d_loss: 0.596176], [ Generator :: loss: 0.932117]
epoch: 19911, [Discriminator :: d_loss: 0.682086], [ Generator :: loss: 0.906965]
epoch: 19912, [Discriminator :: d_loss: 0.694890], [ Generator :: loss: 0.875445]
epoch: 19913, [D